In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt

import sys
sys.path.append("/repos/lnet")
from lnet.utils.plotting import turbo_colormap 

In [ ]:
matplotlib.matplotlib_fname()

In [ ]:
# Save Data
pred_binned = numpy.load("K:/beuttenm/repos/lnet/notebooks/pred_binned.npy")
tgt_binned = numpy.load("K:/beuttenm/repos/lnet/notebooks/tgt_binned.npy")
prs = numpy.load("K:/beuttenm/repos/lnet/notebooks/prs.npy")


In [ ]:
plt.imshow(pred_binned[0].max(0), cmap=turbo_colormap)
plt.savefig("test.ps")

In [ ]:
plt.imshow(tgt_binned[0].max(0), cmap=turbo_colormap)

In [ ]:
plt.imshow(prs[13], cmap=turbo_colormap)
plt.colorbar()
print('pr max', prs.max())

In [ ]:
stds = tgt_binned.std(axis=0)
stds.shape

In [ ]:
print('max', stds.max())
print('mean', stds.mean())
print('med', numpy.median(stds))
print('p80', numpy.percentile(stds, 80))

In [ ]:
mask = stds > numpy.percentile(stds, 80)
masked_prs = numpy.array(prs)
masked_prs[~mask] = numpy.nan
print('mean masked pr', numpy.nanmean(masked_prs))
plt.imshow(mask.max(axis=0))

In [ ]:
msum = mask.sum()
print(msum, msum / numpy.prod(mask.shape))

In [ ]:
def plot_masked(data, mask):
    data = numpy.array(data)
    data[~mask] = numpy.nan
    plt.imshow(numpy.nanmax(data, axis=0))
    plt.colorbar()

In [ ]:
plot_masked(tgt_binned[0], mask)

In [ ]:
plot_masked(prs, mask)

In [ ]:
def plot_ts(tgt, pred, name=""):
    fig, ax_tgt = plt.subplots()
    ax_pred = ax_tgt.twinx()
    ax_tgt.set_xlabel("time [s]")
    ax_tgt.set_ylabel("RL reconstruction [$\Delta F/F_0$]")
    ax_pred.set_ylabel("Network prediction [$\Delta F/F_0$]")
    
    t = numpy.arange(tgt.shape[0]) * 0.11
    tgt = numpy.array(tgt)
    pred = numpy.array(pred)
    add_label = ""
    lns = ax_tgt.plot(t, tgt, color="r", label="RL reconstruction" + add_label)
    lns += ax_pred.plot(t, pred, color="b", label="Network prediction" + add_label) # + f" {pcoeff:.2f}

    labs = [l.get_label() for l in lns]
    ax_tgt.legend(lns, labs)
    if name:
        plt.savefig(f"/repos/lnet/notebooks/{name}.pgf")
    
    plt.show()
    
    
def plot_idx(idx, name=""):
    t_max = None
    plot_ts(tgt_binned[:, mask][:t_max, idx], pred_binned[:, mask][:t_max, idx], name=name)

In [ ]:
# prsidx = numpy.where(prs[mask])[0]
numpy.argsort(prs[mask])
idxs = numpy.argsort(prs[mask])[::-1]

In [ ]:
plot_idx(idxs[0], "test")
idxs[-1]

In [ ]:
for idx in range(0, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
for idx in range(1, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
for idx in range(2, 1000, 100):
    plot_ts(tgt_binned[:, mask][:, idx], pred_binned[:, mask][:, idx])

In [ ]:
t_max = 300
cnt = 0
for idx in range(0, 1000, 10):
    selected_tgt = tgt_binned[:t_max, mask][:, idx]
    selected_pred = pred_binned[:t_max, mask][:, idx]
    
    if selected_tgt[100] < .2:
        continue

    cnt += 1
    if cnt > 10: 
        break
    plot_ts(selected_tgt, selected_pred)

In [ ]:
t_max = 400
cnt = 0
for idx in range(0, 1000, 10):
    selected_tgt = tgt_binned[:t_max, mask][:, idx]
    selected_pred = pred_binned[:t_max, mask][:, idx]
    
    if selected_tgt[100] < .2:
        continue

    cnt += 1
    if cnt > 10: 
        break
    plot_ts(selected_tgt, selected_pred)

In [ ]:
def plot_hist(prs):
    fig, ax = plt.subplots()
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    ax.set_xlim(xmin=0)
    ax.set_xlabel("Pearson Correlation Coefficient")
    ax.set_ylabel("counts")

    plt.hist(prs, bins=25)


In [ ]:
plot_hist(prs[mask])
# plt.savefig("hist.svg")

In [ ]:
prs[mask].mean()

In [ ]:
plot_hist(prs.flatten())

In [ ]:
prs.mean()

In [ ]:

selected_masked = [1256, 1963, 1688, 2365] 

In [ ]:
x, y, z = numpy.where(mask)
for s in selected_masked:
    print(s)
    print(z[s], y[s], x[s])
    
    print('here', s, y[s] / 28, x[s] / 21)
    
print(mask.shape)

In [ ]:
for s in selected_masked:
    plot_idx(s, name=str(s))